In [1]:
import pandas as pd
import os

### 1. 데이터 리스트로 불러오기

In [3]:
dir_list = os.listdir('../data/total_data/')
dir_list

['동별 데이터.xlsx',
 '부산지하철 역사별 행정동.csv',
 '역별 데이터.xlsx',
 '지하철 역사 내 상가 임대 정보.csv',
 '출퇴근시간 유동인구_2021_월.xlsx',
 '출퇴근시간 유동인구_2021_일.xlsx',
 '행정동별_업장의_수.csv']

- 파일 읽어오기

In [4]:
def readfile(file_name):
    f_type = file_name.split('.')[-1]
    if f_type == 'xlsx':
        return pd.read_excel(f'../data/total_data/{file_name}',index_col=0).reset_index(False)
    elif f_type == 'csv':
        return pd.read_csv(f'../data/total_data/{file_name}',encoding='cp949',index_col=0)
    else:
        return None

readfile(dir_list[0])

,구군별,동별,소계,1인가구수
0,중구,중앙동,2435,1963
1,중구,동광동,1703,1122
2,중구,대청동,3562,2036
3,중구,보수동,6029,3431
4,중구,부평동,2748,1702
...,...,...,...,...
200,기장군,기장읍,25757,11540
201,기장군,장안읍,4709,2617
202,기장군,정관읍,32042,9427
203,기장군,일광읍,11772,3744


In [5]:
dong_data = readfile(dir_list[0])
dong_store = readfile(dir_list[-1])

In [6]:
dong_data.head()

,구군별,동별,소계,1인가구수
0,중구,중앙동,2435,1963
1,중구,동광동,1703,1122
2,중구,대청동,3562,2036
3,중구,보수동,6029,3431
4,중구,부평동,2748,1702


In [7]:
dong_store.head()

,동 이름,업장 수
0,남천2동,264
1,당리동,412
2,광복동,985
3,전포1동,597
4,중앙동,734


### 2. 데이터 조인하기

- 동이름 변경

In [8]:
dong_store.rename(columns={'동 이름':'동별'},inplace=True)
dong_store.head()

,동별,업장 수
0,남천2동,264
1,당리동,412
2,광복동,985
3,전포1동,597
4,중앙동,734


In [9]:
step_one = dong_data.join(dong_store.set_index('동별'), on='동별')
step_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구군별     205 non-null    object 
 1   동별      205 non-null    object 
 2   소계      205 non-null    int64  
 3   1인가구수   205 non-null    int64  
 4   업장 수    204 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 8.1+ KB


In [10]:
step_one.head()

,구군별,동별,소계,1인가구수,업장 수
0,중구,중앙동,2435,1963,734.0
1,중구,동광동,1703,1122,80.0
2,중구,대청동,3562,2036,140.0
3,중구,보수동,6029,3431,155.0
4,중구,부평동,2748,1702,370.0


### 3. 지하상가 전체 갯수와 비어있는 상가, 사용중인 상가 갯수를 나타내기 위한 데이터 프레임

- 상가 갯수 추가할 열 생성

In [11]:
station_store = readfile(dir_list[3])
station_store.head()

,상가번호,역사명,상가명,업종,면적(㎡),연간계약금액(부가세포함),계약시작일,계약종료일,비고
0,9501,다대포해수욕장역,CU,브랜드 편의점,52.54,NaN,20170420.0,20220419.0,휴점
1,9502,다대포해수욕장역,상가2호,상가,15.00,15840000.0,20191203.0,20241202.0,NaN
2,9601,다대포항역,CU,브랜드 편의점,75.60,NaN,20170420.0,20220419.0,휴점
3,9701,낫개역,CU,브랜드 편의점,24.33,NaN,20170420.0,20220419.0,NaN
4,9801,신장림역,CU,브랜드 편의점,77.57,NaN,20170420.0,20220419.0,NaN


In [12]:
station_store_num = pd.DataFrame(columns=['역사명','상가 갯수','사용중인 상가','비어있는 상가'])

In [13]:
station_store_num.head()

,역사명,상가 갯수,사용중인 상가,비어있는 상가


In [14]:
# nan값을 0으로 대체하여 공실 여부 판단
station_store.fillna(0,inplace=True)

In [15]:
store_info = {}

for idx in range(len(station_store)):
    item = station_store.iloc[idx]
    if list(item)[1] in store_info.keys():
        store_info[list(item)[1]]['상가 갯수'] += 1
        if list(item)[6] != 0.0:
            store_info[list(item)[1]]['사용중인 상가'] += 1
        else:
            store_info[list(item)[1]]['비어있는 상가'] += 1
    else:
        store_info[list(item)[1]] = {'상가 갯수':1,'사용중인 상가':0,'비어있는 상가':0}
        if list(item)[6] != 0.0:
            store_info[list(item)[1]]['사용중인 상가'] += 1
        else:
            store_info[list(item)[1]]['비어있는 상가'] += 1

In [16]:
len(store_info.keys())

94

In [17]:
i = 0
for item in store_info.keys():
    station_store_num.loc[i] = [item[:-1]]+list(store_info[item].values())
    i+=1

In [18]:
station_store_num.head()

,역사명,상가 갯수,사용중인 상가,비어있는 상가
0,다대포해수욕장,2,2,0
1,다대포항,1,1,0
2,낫개,1,1,0
3,신장림,1,1,0
4,장림,1,1,0


In [19]:
station_store_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 0 to 93
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   역사명      94 non-null     object
 1   상가 갯수    94 non-null     int64 
 2   사용중인 상가  94 non-null     int64 
 3   비어있는 상가  94 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 3.7+ KB


In [ ]:
# station_store_num[station_store_num['상가 갯수']>1]


In [20]:
station_dong = readfile(dir_list[1])
station_dong.head()

,역명,행정동
선명,,
1호선,다대포해수욕장,다대1동
1호선,다대포항,다대1동
1호선,낫개,다대2동
1호선,신장림,장림2동
1호선,장림,장림2동


In [21]:
station_dong.reset_index(drop=True,inplace=True)
station_dong.rename(columns={'역명':'역사명'},inplace=True)
station_dong.head()

,역사명,행정동
0,다대포해수욕장,다대1동
1,다대포항,다대1동
2,낫개,다대2동
3,신장림,장림2동
4,장림,장림2동


In [22]:
station_dong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   역사명     114 non-null    object
 1   행정동     114 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB


In [23]:
station_data = readfile(dir_list[2])
station_data.head()

,호선,역사명,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간
0,1,다대포해수욕장역,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0
1,1,다대포항역,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0
2,1,낫개역,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0
3,1,신장림역,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0
4,1,장림역,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0


In [24]:
import re

In [25]:
for idx in range(len(station_dong.역사명)):
    if '(' in station_dong.역사명.iloc[idx]:
        station_dong.역사명.iloc[idx] = re.split('\(',station_dong.역사명.iloc[idx])[0]

In [26]:
for idx in range(len(station_data.역사명)):
    station_data.역사명.iloc[idx] = station_data.역사명.iloc[idx][:-1]
station_data.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_4384\4229311338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_data.역사명.iloc[idx] = station_data.역사명.iloc[idx][:-1]


,호선,역사명,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간
0,1,다대포해수욕장,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0
1,1,다대포항,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0
2,1,낫개,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0
3,1,신장림,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0
4,1,장림,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0


In [27]:
print(list(station_dong.역사명))
print(list(station_data.역사명))

['다대포해수욕장', '다대포항', '낫개', '신장림', '장림', '동매', '신평', '하단', '당리', '사하', '괴정', '대티', '서대신', '동대신', '토성', '자갈치', '남포', '중앙', '부산', '초량', '부산진', '좌천', '범일', '범내골', '서면', '부전', '양정', '시청', '연산', '교대', '동래', '명륜', '온천장', '부산대', '장전', '구서', '두실', '남산', '범어사', '노포', '장산', '중동', '해운대', '동백', '벡스코', '센텀시티', '민락', '수영', '광안', '금련산', '남천', '경성대·부경대', '대연', '못골', '지게골', '문현', '국제금융센터·부산은행', '전포', '서면', '부암', '가야', '동의대', '개금', '냉정', '주례', '감전', '사상', '덕포', '모덕', '모라', '구남', '구명', '덕천', '수정', '화명', '율리', '동원', '금곡', '호포', '증산', '부산대양산캠퍼스', '남양산', '양산', '수영', '망미', '배산', '물만골', '연산', '거제', '종합운동장', '사직', '미남', '만덕', '남산정', '숙등', '덕천', '구포', '강서구청', '체육공원', '대저', '미남', '동래', '수안', '낙민', '충렬사', '명장', '서동', '금사', '반여농산물시장', '석대', '영산대', '동부산대학', '고촌', '안평']
['다대포해수욕장', '다대포항', '낫개', '신장림', '장림', '동매', '신평', '하단', '당리', '사하', '괴정', '대티', '서대신', '동대신', '토성', '자갈치', '남포', '중앙', '부산', '초량', '부산진', '좌천', '범일', '범내골', '서면', '부전', '양정', '시청', '연산', '교대', '동래', '명륜', '온천장', '부산대', '장전', '구서', '두실', '남산', '범어사', '

- 데이터 변경
    - 경성부경대 -> 경성대·부경대
    - 미남, 수영 제거

In [43]:
station_data.isnull().sum()

호선           0
역사명          0
환승         102
역위도          0
역경도          0
일평균승차        0
일평균하차        0
일평균승하차       0
순위           0
하행 대기시간      0
상행 대기시간      0
dtype: int64

In [44]:
station_data[station_data['순위'].isnull()]

,호선,역사명,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간


In [45]:
station_data = station_data.drop([83,100]).reset_index(drop=True)
station_data

,호선,역사명,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간
0,1,다대포해수욕장,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0
1,1,다대포항,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0
2,1,낫개,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0
3,1,신장림,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0
4,1,장림,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...
105,4,석대,NaN,35.218112,129.137179,195.0,151.0,346.0,112.0,20.0,20.0
106,4,영산대,NaN,35.225777,129.146149,2419.0,2377.0,4796.0,95.0,20.0,20.0
107,4,동부산대학,NaN,35.232506,129.154024,2223.0,2097.0,4320.0,97.0,20.0,20.0
108,4,고촌,NaN,35.236031,129.160444,569.0,447.0,1016.0,111.0,20.0,20.0


In [46]:
station_data['역사명'] = station_data['역사명'].replace(['경성부경대'],['경성대·부경대'])
station_data[station_data['역사명'].isin(['경성대·부경대'])]

,호선,역사명,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간
51,2,경성대·부경대,NaN,35.137585,129.100548,11892.0,12503.0,24395.0,14.0,30.0,30.0


In [47]:
print(len(list(station_dong.역사명)))
print(len(list(station_data.역사명)))

114
110


In [48]:
step_two = station_dong.join(station_data.set_index('역사명'),on='역사명')
step_two

,역사명,행정동,호선,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간
0,다대포해수욕장,다대1동,1.0,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0
1,다대포항,다대1동,1.0,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0
2,낫개,다대2동,1.0,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0
3,신장림,장림2동,1.0,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0
4,장림,장림2동,1.0,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...
109,석대,반송1동,4.0,NaN,35.218112,129.137179,195.0,151.0,346.0,112.0,20.0,20.0
110,영산대,반송1동,4.0,NaN,35.225777,129.146149,2419.0,2377.0,4796.0,95.0,20.0,20.0
111,동부산대학,반송2동,4.0,NaN,35.232506,129.154024,2223.0,2097.0,4320.0,97.0,20.0,20.0
112,고촌,고촌리,4.0,NaN,35.236031,129.160444,569.0,447.0,1016.0,111.0,20.0,20.0


In [49]:
step_two = step_two.join(station_store_num.set_index('역사명'),on='역사명')
step_two.head()

,역사명,행정동,호선,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간,상가 갯수,사용중인 상가,비어있는 상가
0,다대포해수욕장,다대1동,1.0,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0,2.0,2.0,0.0
1,다대포항,다대1동,1.0,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0,1.0,1.0,0.0
2,낫개,다대2동,1.0,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0,1.0,1.0,0.0
3,신장림,장림2동,1.0,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0,1.0,1.0,0.0
4,장림,장림2동,1.0,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0,1.0,1.0,0.0


In [50]:
step_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구군별     205 non-null    object 
 1   동별      205 non-null    object 
 2   소계      205 non-null    int64  
 3   1인가구수   205 non-null    int64  
 4   업장 수    204 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 8.1+ KB


In [51]:
step_two.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122 entries, 0 to 113
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   역사명      122 non-null    object 
 1   행정동      122 non-null    object 
 2   호선       120 non-null    float64
 3   환승       20 non-null     float64
 4   역위도      120 non-null    float64
 5   역경도      120 non-null    float64
 6   일평균승차    120 non-null    float64
 7   일평균하차    120 non-null    float64
 8   일평균승하차   120 non-null    float64
 9   순위       120 non-null    float64
 10  하행 대기시간  120 non-null    float64
 11  상행 대기시간  120 non-null    float64
 12  상가 갯수    81 non-null     float64
 13  사용중인 상가  81 non-null     float64
 14  비어있는 상가  81 non-null     float64
dtypes: float64(13), object(2)
memory usage: 15.2+ KB


In [52]:
step_two.rename(columns={'행정동':'동별'},inplace=True)

In [53]:
final_dataframe = step_two.join(step_one.set_index('동별'),on='동별')

In [54]:
final_dataframe.head()

,역사명,동별,호선,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간,상가 갯수,사용중인 상가,비어있는 상가,구군별,소계,1인가구수,업장 수
0,다대포해수욕장,다대1동,1.0,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0,2.0,2.0,0.0,사하구,16941.0,6298.0,345.0
1,다대포항,다대1동,1.0,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0,1.0,1.0,0.0,사하구,16941.0,6298.0,345.0
2,낫개,다대2동,1.0,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0,1.0,1.0,0.0,사하구,12110.0,4651.0,246.0
3,신장림,장림2동,1.0,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0,1.0,1.0,0.0,사하구,11905.0,4165.0,302.0
4,장림,장림2동,1.0,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0,1.0,1.0,0.0,사하구,11905.0,4165.0,302.0


In [55]:
print(list(step_one.동별))
print(list(step_two.동별))

['중앙동', '동광동', '대청동', '보수동', '부평동', '광복동', '남포동', '영주1동', '영주2동', '동대신1동', '동대신2동', '동대신3동', '서대신1동', '서대신3동', '서대신4동', '부민동', '아미동', '초장동', '충무동', '남부민1동', '남부민2동', '암남동', '초량1동', '초량2동', '초량3동', '초량6동', '수정1동', '수정2동', '수정4동', '수정5동', '좌천동', '범일1동', '범일2동', '범일5동', '남항동', '영선1동', '영선2동', '신선동', '봉래1동', '봉래2동', '청학1동', '청학2동', '동삼1동', '동삼2동', '동삼3동', '부전1동', '부전2동', '연지동', '초읍동', '양정1동', '양정2동', '전포1동', '전포2동', '부암1동', '부암3동', '당감1동', '당감2동', '당감4동', '가야1동', '가야2동', '개금1동', '개금2동', '개금3동', '범천1동', '범천2동', '수민동', '복산동', '명륜동', '온천1동', '온천2동', '온천3동', '사직1동', '사직2동', '사직3동', '안락1동', '안락2동', '명장1동', '명장2동', '대연1동', '대연3동', '대연4동', '대연5동', '대연6동', '용호1동', '용호2동', '용호3동', '용호4동', '용당동', '감만1동', '감만2동', '우암동', '문현1동', '문현2동', '문현3동', '문현4동', '구포1동', '구포2동', '구포3동', '금곡동', '화명1동', '화명2동', '화명3동', '덕천1동', '덕천2동', '덕천3동', '만덕1동', '만덕2동', '만덕3동', '우1동', '우2동', '우3동', '중1동', '중2동', '좌1동', '좌2동', '좌3동', '좌4동', '송정동', '반여1동', '반여2동', '반여3동', '반여4동', '반송1동', '반송2동', '재송1동', '재송2동', '괴정1동', '괴정2동', 

In [56]:
final_dataframe.to_csv('../data/total_data/데이터_합본.csv',encoding='cp949')

In [57]:
final_dataframe = pd.read_csv('../data/total_data/데이터_합본.csv',encoding='cp949',index_col=0)
final_dataframe.head()

,역사명,동별,호선,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간,상가 갯수,사용중인 상가,비어있는 상가,구군별,소계,1인가구수,업장 수
0,다대포해수욕장,다대1동,1.0,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0,2.0,2.0,0.0,사하구,16941.0,6298.0,345.0
1,다대포항,다대1동,1.0,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0,1.0,1.0,0.0,사하구,16941.0,6298.0,345.0
2,낫개,다대2동,1.0,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0,1.0,1.0,0.0,사하구,12110.0,4651.0,246.0
3,신장림,장림2동,1.0,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0,1.0,1.0,0.0,사하구,11905.0,4165.0,302.0
4,장림,장림2동,1.0,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0,1.0,1.0,0.0,사하구,11905.0,4165.0,302.0


In [58]:
final_dataframe.head()

,역사명,동별,호선,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간,상가 갯수,사용중인 상가,비어있는 상가,구군별,소계,1인가구수,업장 수
0,다대포해수욕장,다대1동,1.0,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0,2.0,2.0,0.0,사하구,16941.0,6298.0,345.0
1,다대포항,다대1동,1.0,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0,1.0,1.0,0.0,사하구,16941.0,6298.0,345.0
2,낫개,다대2동,1.0,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0,1.0,1.0,0.0,사하구,12110.0,4651.0,246.0
3,신장림,장림2동,1.0,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0,1.0,1.0,0.0,사하구,11905.0,4165.0,302.0
4,장림,장림2동,1.0,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0,1.0,1.0,0.0,사하구,11905.0,4165.0,302.0


In [59]:
step_one.to_csv('../data/total_data/동_데이터_합본.csv',encoding='cp949')
step_two.to_csv('../data/total_data/역_데이터_합본.csv',encoding='cp949')

In [60]:
new_step_two = pd.read_csv('../data/total_data/역_데이터_합본.csv',encoding='cp949',index_col=0)
new_step_two.head()

,역사명,동별,호선,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간,상가 갯수,사용중인 상가,비어있는 상가
0,다대포해수욕장,다대1동,1.0,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0,2.0,2.0,0.0
1,다대포항,다대1동,1.0,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0,1.0,1.0,0.0
2,낫개,다대2동,1.0,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0,1.0,1.0,0.0
3,신장림,장림2동,1.0,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0,1.0,1.0,0.0
4,장림,장림2동,1.0,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0,1.0,1.0,0.0


In [61]:
total_df = new_step_two.join(step_one.set_index('동별'),on='동별')
total_df.head()

,역사명,동별,호선,환승,역위도,역경도,일평균승차,일평균하차,일평균승하차,순위,하행 대기시간,상행 대기시간,상가 갯수,사용중인 상가,비어있는 상가,구군별,소계,1인가구수,업장 수
0,다대포해수욕장,다대1동,1.0,NaN,35.048670,128.964100,4264.0,3497.0,7761.0,69.0,20.0,0.0,2.0,2.0,0.0,사하구,16941.0,6298.0,345.0
1,다대포항,다대1동,1.0,NaN,35.057820,128.971300,2900.0,3321.0,6221.0,80.0,20.0,20.0,1.0,1.0,0.0,사하구,16941.0,6298.0,345.0
2,낫개,다대2동,1.0,NaN,35.065265,128.979873,4104.0,4223.0,8327.0,64.0,20.0,20.0,1.0,1.0,0.0,사하구,12110.0,4651.0,246.0
3,신장림,장림2동,1.0,NaN,35.074433,128.977041,2777.0,2583.0,5360.0,91.0,20.0,20.0,1.0,1.0,0.0,사하구,11905.0,4165.0,302.0
4,장림,장림2동,1.0,NaN,35.081090,128.977500,2854.0,2636.0,5490.0,89.0,20.0,20.0,1.0,1.0,0.0,사하구,11905.0,4165.0,302.0


In [ ]:
total_df.to_csv('../data/total_data/데이터_합본.csv',encoding='cp949')